## E4 - Heuristic Recursive Sampling in 2D 
Experiment 4 used the initial conditions of experiment 3, utilizing a set of 1,000 'neutralized' proteins for lengths {5,10,15,...,95,100}, folded on a 2D lattice. In this experiment, however, we diverged from stochastic methods by introducing a more deterministic backtracking method. Upon reaching an impasse where no further placements were valid, the deterministic approach used a heuristic backtracking rule that can be expressed using propositional logic:

$$
L \lor (\neg L \land M) \lor (\neg L \land \neg M \land R)
$$

This rule states that:
- **left** is the first preference.
- If **left** fails, then try **middle** as the next option.
- If both **left** and **middle** fail  then **right** is the final option for that recursive call.


![Example Screenshot](5-methods.png)

The backtracking continued in this structured manner, systematically exploring the next available direction in the predetermined sequence whenever an impasse was encountered. The recursive algorithm ensured adherence to the sequence of directions predetermined for each amino acid, retracing steps in a heuristic manner whenever the folding path reached a dead end. 

In [21]:
import numpy as np 
import pandas as pd
import time as time
import matplotlib.pyplot as plt
np.set_printoptions(threshold=np.inf)

### Step 1 - Initializing Acids

This function `random_division` takes the total number of amino acids as input and randomly divides them into hydrophobic and polar amino acids, before creating a random 1D amino acid string. It ensures that there's at least one hydrophobic and one polar amino acid. You can adjust the total_amino_acids variable to the desired number of amino acids you want to divide.

In [2]:
total_amino_acids = 12 # Change this to a desired number, 10 is used for illustration purposes

def random_division(total_amino_acids):
    num_hydrophobic = np.random.randint(1, total_amino_acids)  # Ensure at least one H and P
    num_polar = total_amino_acids - num_hydrophobic
    # randomly shuffle the amino acids
    amino_acids = ['H'] * num_hydrophobic + ['P'] * num_polar
    np.random.shuffle(amino_acids)
    return num_hydrophobic, num_polar, amino_acids

num_hydrophobic, num_polar, amino_acids = random_division(total_amino_acids)
print("Number of Hydro acids:", num_hydrophobic)
print("Number of Polar acids:", num_polar)
print("Random Amino Acid String:", "".join(amino_acids))

Number of Hydro acids: 2
Number of Polar acids: 10
Random Amino Acid String: HPPPPPPHPPPP


### Step 2 - Initializing Grid
The `initialize_grid` function generates a grid to place amino acids, with specified dimensions, and populates it with H and P amino acids, while ensuring they do not overlap and tracks their placement order. This allows for simulating the random arrangement of amino acids on a grid for various applications.

In [4]:
def determine_direction(prev_row, prev_col, current_row, current_col):
    """
    Determines the cardinal direction of the current move in comparison to the previous move.
    """
    # Determine the previous and current movement vectors
#     prev_move_vector = (prev_row - prev_prev_row, prev_col - prev_prev_col)
    current_move_vector = (current_row - prev_row, current_col - prev_col)

    # Define movement vectors for comparison
    movement_vectors = {
        'up': (-1, 0),
        'down': (1, 0),
        'left': (0, -1),
        'right': (0, 1)
    }

    # Compare the current movement vector with the cardinal directions
    for direction, vector in movement_vectors.items():
        if current_move_vector == vector:
            return direction

    # Default return if no direction matches
    return None


In [5]:
def get_deterministic_neighbors(last_row, last_col, relative_direction):
    """
    Determine neighbors based on a "left, middle, right" heuristic, always prioritizing left.
    The 'relative_direction' parameter indicates the direction relative to the last move.
    """
    if relative_direction == 'down':
        # Moving down: left is right on the grid, right is left on the grid
        left = (last_row, last_col + 1)  # Right on the grid
        middle = (last_row + 1, last_col)  # Down
        right = (last_row, last_col - 1)  # Left on the grid
        
    elif relative_direction == 'up':
        # Moving up: left is left on the grid, right is right on the grid
        left = (last_row, last_col - 1)  # Left on the grid
        middle = (last_row - 1, last_col)  # Up
        right = (last_row, last_col + 1)  # Right on the grid
        
    elif relative_direction == 'left':
        # Moving left: left is down on the grid, right is up on the grid
        left = (last_row + 1, last_col)  # Down
        middle = (last_row, last_col - 1)  # Left
        right = (last_row - 1, last_col)  # Up
        
    else:  # relative_direction == 'right'
        # Moving right: left is up on the grid, right is down on the grid
        left = (last_row - 1, last_col)  # Up
        middle = (last_row, last_col + 1)  # Right
        right = (last_row + 1, last_col)  # Down

    neighbors = [left, middle, right]
    return neighbors

In [6]:
import numpy as np

def place_amino_acid(grid, amino_acids_copy, amino_acid_order, backtrack_count, last_amino_row=None, last_amino_col=None, we_just_backtracked=0, fenced=None, fence_limit=0, fences=0, acids_placed_since_backtrack=0):
    grid_size = len(grid)
    
    # Base case: if no amino acids left, return True
    if not amino_acids_copy:
        return True, backtrack_count
    
    # For the first amino acid
    if last_amino_row is None or last_amino_col is None:
        first_amino_row = grid_size // 2
        first_amino_col = grid_size // 2
        amino_acid_type = amino_acids_copy.pop(0)
        grid[first_amino_row, first_amino_col] = amino_acid_type
        amino_acid_order.append((amino_acid_type, (first_amino_row, first_amino_col)))
        return place_amino_acid(grid, amino_acids_copy, amino_acid_order, backtrack_count, first_amino_row, first_amino_col, we_just_backtracked, fenced, fence_limit, fences, acids_placed_since_backtrack)

    # Handling all other acids
    neighbors = [
        (last_amino_row - 1, last_amino_col),
        (last_amino_row + 1, last_amino_col),
        (last_amino_row, last_amino_col - 1),
        (last_amino_row, last_amino_col + 1)
    ]
    
    valid_neighbors = [(row, col) for row, col in neighbors if 0 <= row < grid_size and 0 <= col < grid_size and grid[row, col] == '']
    np.random.shuffle(valid_neighbors)
    
    amino_acid_type = amino_acids_copy.pop(0)
    if len(valid_neighbors) > 0:
        grid[valid_neighbors[0][0], valid_neighbors[0][1]] = amino_acid_type
        amino_acid_order.append((amino_acid_type, (valid_neighbors[0][0], valid_neighbors[0][1])))
        acids_placed_since_backtrack += 1  # Increment acid placement counter
        return place_amino_acid(grid, amino_acids_copy, amino_acid_order, backtrack_count, valid_neighbors[0][0], valid_neighbors[0][1], 0, fenced, fence_limit, fences, acids_placed_since_backtrack)  # Recursive call

    else:
        # Backtracking logic
        backtrack_count += 1
        we_just_backtracked = 1
        fence_limit += 1
        amino_acids_copy.insert(0, amino_acid_type)
        grid[last_amino_row, last_amino_col] = ''
        amino_acid_order.pop()
        
        # Remove fences if the number of acids placed exceeds the number of fences
        if acids_placed_since_backtrack > fences:
            # Remove all fences
            for i in range(grid_size):
                for j in range(grid_size):
                    if grid[i, j] == 'F':
                        grid[i, j] = ''
            fences = 0  # Reset fence count

        # Reset acid placement counter after backtracking
        acids_placed_since_backtrack = 0
        
        if not amino_acid_order:  # If all acids have been removed, return failure
            return False, backtrack_count

        # Repeat the process for the last acid in the order
        _, (prev_row, prev_col) = amino_acid_order[-1]
        return place_amino_acid(grid, amino_acids_copy, amino_acid_order, backtrack_count, prev_row, prev_col, we_just_backtracked, fenced, fence_limit, fences, acids_placed_since_backtrack)

    # If no neighbors lead to a solution, return False
    return False, backtrack_count


In [7]:
counter = 0
amino_acids_copy = amino_acids.copy()
def initialize_grid(amino_acids, num_hydrophobic, num_polar):
    amino_acids_copy = amino_acids.copy()
    total_amino_acids = len(amino_acids)
    grid_size = total_amino_acids * 2
    grid = np.full((grid_size, grid_size), fill_value='', dtype=object)
    amino_acid_order = []

    is_placed, backtrack_count = place_amino_acid(grid, amino_acids_copy, amino_acid_order, 0)
#     plot_amino_acid_order(amino_acid_order)

    return grid, amino_acid_order, backtrack_count

# Example usage
initial_grid, amino_acid_order, backtrack_count = initialize_grid(amino_acids_copy, num_hydrophobic, num_polar)

In [8]:
print(f"Total backtracks: {backtrack_count}")

Total backtracks: 0


In [9]:
print("Amino Acid Order:", amino_acid_order)

Amino Acid Order: [('H', (12, 12)), ('P', (13, 12)), ('P', (13, 11)), ('P', (14, 11)), ('P', (14, 10)), ('P', (15, 10)), ('P', (15, 11)), ('H', (15, 12)), ('P', (14, 12)), ('P', (14, 13)), ('P', (15, 13)), ('P', (16, 13))]


In [10]:
print(initial_grid)

[['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' 'H' '' '' '' '' '' '' '' '' '' ''
  '']
 ['' '' '' '' '' '' ''

In [11]:
def determine_directions(amino_acid_order):
    """
    Determines the direction of each amino acid placement relative to the previous one.
    Directions are 'left', 'straight', 'right', considering the orientation of the movement from the previous point.
    """
    directions = ['start']  # First amino acid has no direction

    # Define movement vectors for easier comparison
    movement_vectors = {
        'up': (-1, 0),
        'down': (1, 0),
        'left': (0, -1),
        'right': (0, 1)
    }

    for i in range(1, len(amino_acid_order)):
        # Get the current and previous amino acid's row and column
        _, (current_row, current_col) = amino_acid_order[i]
        _, (prev_row, prev_col) = amino_acid_order[i - 1]

        # Determine the movement vector from the previous amino acid
        move_vector = (current_row - prev_row, current_col - prev_col)

        if i == 1:
            # For the second amino acid, we don't have a previous direction, so we set it as straight
            direction = 'straight'
        else:
            # Get the previous movement vector
            _, (prev_prev_row, prev_prev_col) = amino_acid_order[i - 2]
            prev_move_vector = (prev_row - prev_prev_row, prev_col - prev_prev_col)

            # Determine direction based on previous movement vector
            if prev_move_vector in [movement_vectors['up'], movement_vectors['down']]:
                # Moving vertically
                if move_vector == movement_vectors['left']:
                    direction = 'left' if prev_move_vector == movement_vectors['up'] else 'right'
                elif move_vector == movement_vectors['right']:
                    direction = 'right' if prev_move_vector == movement_vectors['up'] else 'left'
                else:
                    direction = 'straight'
            else:
                # Moving horizontally
                if move_vector == movement_vectors['up']:
                    direction = 'left' if prev_move_vector == movement_vectors['right'] else 'right'
                elif move_vector == movement_vectors['down']:
                    direction = 'right' if prev_move_vector == movement_vectors['right'] else 'left'
                else:
                    direction = 'straight'

        directions.append(direction)

    return directions

# Test the refined function with the provided example amino_acid_order
determine_directions(amino_acid_order)

['start',
 'straight',
 'right',
 'left',
 'right',
 'left',
 'left',
 'straight',
 'left',
 'right',
 'right',
 'straight']

In [12]:
def trim_empty_rows_and_columns(grid):
    # Find the indices of non-empty rows and columns
    non_empty_rows = np.any(grid != '', axis=1)
    non_empty_columns = np.any(grid != '', axis=0)

    # Use boolean indexing to extract non-empty rows and columns
    trimmed_grid = grid[non_empty_rows][:, non_empty_columns]

    return trimmed_grid

# Call the function to trim empty rows and columns
trimmed_grid = trim_empty_rows_and_columns(initial_grid)

# Print the trimmed grid
print("Trimmed Grid:", '\n', trimmed_grid)

Trimmed Grid: 
 [['' '' 'H' '']
 ['' 'P' 'P' '']
 ['P' 'P' 'P' 'P']
 ['P' 'P' 'H' 'P']
 ['' '' '' 'P']]


### Step 3a - Calculate H-bonds
The `find_H_combinations_grid` function is used to~ identify and collect sets of coordinates representing adjacent 'H' amino acids in a grid. It iterates through the entire grid, checking each position for the presence of 'H' amino acids. If an 'H' amino acid is found, it examines neighboring positions (up, down, left, right) to identify adjacent 'H' amino acids. For each pair of adjacent 'H' amino acids, it creates a frozenset containing their coordinates (ensuring that the order of coordinates doesn't matter) and adds this frozenset to a set. This set stores all unique pairs of adjacent 'H' amino acids found in the grid. The function returns this set of adjacent 'H' amino acid pairs.

In [13]:
def find_H_pairs_grid(grid):
    adjacent_hydrophobic_amino_acids = set()  # Use a set to automatically remove duplicates

    # Iterate through the grid to find adjacent 'H' amino acids
    for row in range(grid.shape[0]):
        for col in range(grid.shape[1]):
            current_acid = grid[row, col]

            # Check if the current amino acid is 'H'
            if current_acid == 'H':
                # Check the neighboring positions (up, down, left, right) relative to the current position
                neighbors = [
                    (row - 1, col),
                    (row + 1, col),
                    (row, col - 1),
                    (row, col + 1)
                ]

                for neighbor_row, neighbor_col in neighbors:
                    # Check if the neighbor is within the grid bounds
                    if 0 <= neighbor_row < grid.shape[0] and 0 <= neighbor_col < grid.shape[1]:
                        neighbor_acid = grid[neighbor_row, neighbor_col]

                        # Check if the neighbor is also 'H'
                        if neighbor_acid == 'H':
                            # Use frozenset to ensure that the order of coordinates doesn't matter
                            amino_acid_pair = frozenset({(row, col), (neighbor_row, neighbor_col)})
                            adjacent_hydrophobic_amino_acids.add(amino_acid_pair)
                            
    return adjacent_hydrophobic_amino_acids

find_H_pairs_grid(initial_grid)

set()

### Step 3b - Calculate H-bonds
The `find_H_combinations_order` function examines the order of amino acids and identifies adjacent 'H' amino acids. It does this by iterating through the amino acid order, checking pairs of consecutive amino acids for 'H' type, and recording these pairs as frozensets in a set to remove duplicates. This function helps identify adjacent 'H' amino acids in the sequence order, which is useful for analyzing the arrangement of amino acids.

In [14]:
def find_H_pairs_order(amino_acid_order):
    adjacent_hydrophobic_amino_acids = set()  # Use a set to automatically remove duplicates

    # Iterate through the amino acid order to find adjacent 'H' amino acids
    for i in range(len(amino_acid_order) - 1):
        current_acid, current_position = amino_acid_order[i]
        next_acid, next_position = amino_acid_order[i + 1]

        # Check if both current and next amino acids are 'H'
        if current_acid == 'H' and next_acid == 'H':
            # Use frozenset to ensure that the order of positions doesn't matter
            amino_acid_pair = frozenset({current_position, next_position})
            adjacent_hydrophobic_amino_acids.add(amino_acid_pair)

    return adjacent_hydrophobic_amino_acids

find_H_pairs_order(amino_acid_order)

set()

### Step 3c - Calculate H-bonds
The provided code calculates the H-combinations in two different ways and then compares them. It calculates the H-pairs in the grid using the `find_H_combinations_grid` function and the H-pairs in the amino acid order using the `find_H_combination_order` function. Then, it subtracts the H-combinations found in the amino acid order from those found in the grid. This comparison helps identify the **H-bonds** that are formed between adjacent 'H' amino acids in the grid but not in the given amino acid sequence order.

In [15]:
def find_H_bonds(grid, amino_acid_order):
    grid_h_pairs = find_H_pairs_grid(grid)
    order_h_pairs = find_H_pairs_order(amino_acid_order)
    return grid_h_pairs - order_h_pairs

H_bonds = find_H_bonds(initial_grid, amino_acid_order)

In [16]:
print('Number of H-bonds:', len(H_bonds))

for bond in H_bonds:
    coordinates = [coord for coord in bond]
    print(coordinates)

Number of H-bonds: 0


### Step 4 - Creating Samples
The code generates N random protein grid configurations with a specified number of hydrophobic and polar amino acids using the `generate_random_samples` function. 

In [17]:
def generate_random_samples(N, total_amino_acids):
    random_samples = []

    for _ in range(N):
        num_hydrophobic, num_polar, amino_acids = random_division(total_amino_acids)
        amino_acids_copy = amino_acids.copy()
        initial_grid, amino_acid_order, backtrack_count = initialize_grid(amino_acids_copy, num_hydrophobic, num_polar)
        trimmed_grid = trim_empty_rows_and_columns(initial_grid)
        protein_dimensions = trimmed_grid.shape
        amino_acids_on_grid = np.count_nonzero(initial_grid != '')
        amino_acids_directions =  determine_directions(amino_acid_order)
        hbonds = len(find_H_bonds(initial_grid, amino_acid_order))
        hratio = hbonds / amino_acids_on_grid
        backtracks = backtrack_count
        
        
        random_samples.append((num_hydrophobic,
                               num_polar,
                               amino_acids,
                               initial_grid,
                               amino_acids_on_grid,
                               amino_acid_order,
                               amino_acids_directions,
                               trimmed_grid, 
                               protein_dimensions,
                               hbonds,
                               hratio,
                               backtracks,
                              ))

    return random_samples

# Example usage
generate_random_samples(3, total_amino_acids)[0]

(7,
 5,
 ['H', 'P', 'H', 'P', 'P', 'P', 'H', 'H', 'H', 'P', 'H', 'H'],
 array([['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
         '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
         '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
         '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
         '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
         '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
         '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
         '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
         '', '', '', '', '', '', '', ''],
        [

In [18]:
import time

def store_samples_in_dataframe(num_samples, amino_acid_lengths):
    
    data = {
        "Amino Acid Length": [],
        "Num Hydrophobic": [],
        "Num Polar": [],
        "1D protein": [],
        "2D protein": [],
        "Amino Acids on Grid": [],  
        "Trimmed 2D protein": [],
        "Shape 2D protein": [],
        "Amino Acid Order": [],
        "Amino Acid Direction": [],
        "H-Bonds": [],
        "H-Ratio": [],
        "Recursions": [],
        "Time Taken (s)": [], 
    }

    for length in amino_acid_lengths:
        random_samples = generate_random_samples(num_samples, length)
        for sample in random_samples:
            start_time = time.time()  # Start time before producing the sample
            
            num_hydrophobic, num_polar, amino_acids, initial_grid, amino_acids_on_grid, amino_acid_order, determine_directions ,trimmed_grid, protein_dimensions, hbonds, hratio, backtracks = sample
            
            end_time = time.time()  # End time after producing the sample
            
            data["Amino Acid Length"].append(length)
            data["Num Hydrophobic"].append(num_hydrophobic)
            data["Num Polar"].append(num_polar)
            data["1D protein"].append(amino_acids)
            data["2D protein"].append(initial_grid)
            data["Amino Acids on Grid"].append(amino_acids_on_grid)
            data["Trimmed 2D protein"].append(trimmed_grid)
            data["Shape 2D protein"].append(protein_dimensions)
            data["Amino Acid Order"].append(amino_acid_order)
            data["Amino Acid Direction"].append(determine_directions),
            data["H-Bonds"].append(hbonds)
            data['H-Ratio'].append(hratio)
            data["Recursions"].append(backtracks)
            data["Time Taken (s)"].append(end_time - start_time)

    df = pd.DataFrame(data)
    return df

In [19]:
def timing_samples(num_samples, amino_acid_lengths):
    total_times = {}

    for length in amino_acid_lengths:
        start_time = time.time()
        
        _ = store_samples_in_dataframe(num_samples, [length])  # We call the original function here
        
        end_time = time.time()
        
        total_times[length] = end_time - start_time
        print(f"Time taken to create and process {num_samples} samples for amino acid length {length}: {total_times[length]:.2f} seconds")

    return total_times

In [20]:
%%time
# Generate an experiment with 1000 samples for amino length [5, 10, 15 ..., X]

num_samples = 1000
amino_acid_lengths = [length for length in range(5, 26, 5)]
time_data = timing_samples(num_samples, amino_acid_lengths)
HP25E2 = store_samples_in_dataframe(num_samples, amino_acid_lengths)

# num_samples = 1000
# amino_acid_lengths = [length for length in range(5, 51, 5)]
# time_data = timing_samples(num_samples, amino_acid_lengths)
# HP50E2 = store_samples_in_dataframe(num_samples, amino_acid_lengths)

# num_samples = 1000
# amino_acid_lengths = [length for length in range(5, 101, 5)]
# time_data = timing_samples(num_samples, amino_acid_lengths)
# HP100E2 = store_samples_in_dataframe(num_samples, amino_acid_lengths)

# num_samples = 1000
# amino_acid_lengths = [length for length in range(5, 201, 5)]
# time_data = timing_samples(num_samples, amino_acid_lengths)
# HP200E2 = store_samples_in_dataframe(num_samples, amino_acid_lengths)



Time taken to create and process 1000 samples for amino acid length 5: 0.11 seconds
Time taken to create and process 1000 samples for amino acid length 10: 0.20 seconds
Time taken to create and process 1000 samples for amino acid length 15: 0.29 seconds
Time taken to create and process 1000 samples for amino acid length 20: 0.44 seconds
Time taken to create and process 1000 samples for amino acid length 25: 0.66 seconds
Time taken to create and process 1000 samples for amino acid length 30: 0.97 seconds
Time taken to create and process 1000 samples for amino acid length 35: 1.59 seconds
Time taken to create and process 1000 samples for amino acid length 40: 2.38 seconds
Time taken to create and process 1000 samples for amino acid length 45: 3.62 seconds
Time taken to create and process 1000 samples for amino acid length 50: 5.41 seconds
CPU times: user 30.5 s, sys: 295 ms, total: 30.8 s
Wall time: 30.8 s


In [ ]:
# HP25E2.to_csv('../Data/Experiment 4/HP25.csv', index=False)

In [22]:
# HP50E2.to_csv('../Data/Experiment 4/HP50.csv', index=False)

In [ ]:
# HP100E2.to_csv('../Data/Experiment 4/HP100.csv', index=False)

In [ ]:
# HP200E2.to_csv('../Data/Experiment 4/HP200.csv', index=False)